In [1]:

# 载入库
import xlrd
import xlwt
from xlwt import Workbook
import numpy as np


In [2]:

# 载入原始Excel表
loc = ('test.xlsx')


In [3]:

# 打开表格及工作簿
wb = xlrd.open_workbook(loc)
sheet = wb.sheet_by_index(0)


In [4]:

# 行数
row = sheet.nrows

# 载入膜号列数
col_of_index = [2]



## TODO:find failed numbers before append them to d[]

In [5]:

# 数据处理、转置、编组、筛选出不合格膜号
# 这里仅仅筛选出位置前移和绝对值偏低的
d = []
locfailed = set()
absfailed = set()
index = -1
downT = 3.95
upT = 5.05
absT = 0.045
absC = 0.7
el = 1 # 区分初筛、复筛
er = 0
for j in col_of_index:
    for i in range(row):
        cur = sheet.cell_value(i, j)
        if type(cur) == float:
            T = float(sheet.cell_value(i, j + 1))
            C = float(sheet.cell_value(i, j + 2))
            t = float(sheet.cell_value(i, j + 3))
            c = float(sheet.cell_value(i, j + 4))
            rValue = sheet.cell_value(i, j + 5)
            er = 0
            if T <= downT or T >= upT:# 筛选T位置偏移
                er += 1
                if er >= el:
                    locfailed.add(int(cur))
            if t <= absT:# 筛选T绝对值低
                er += 1
                if er >= el:
                    absfailed.add(int(cur))
            if c <= absC:# 筛选C绝对值低
                er += 1
                if er >= el:
                    absfailed.add(int(cur))
            if type(rValue) == float or type(rValue) == int:
                if index < 0 or d[index][0] != int(cur):
                    d.append((int(cur), [rValue]))
                    index += 1
                else:
                    d[index][1].append(rValue)    
                    

In [6]:

# 出错膜号
s = set()
wrong_pos = set()
less_pos = set()
for i in range(index + 1):
    if d[i][0] in s:
        wrong_pos.add(d[i][0])
    else:
        s.add(d[i][0])

# 将处理好的数据保存
# 创建工作簿
wb = Workbook()
sheet1 = wb.add_sheet('数据转置', cell_overwrite_ok=True)
sheet2 = wb.add_sheet('初筛处理结果', cell_overwrite_ok=True)

# 标色出错膜号
styleWrongPos = xlwt.easyxf('pattern: pattern solid, fore_colour ocean_blue; font: bold on;')
styleMoreOrLess = xlwt.easyxf('pattern: pattern solid, fore_colour red; font: bold on;')

# 写入抬头
sheet1.write(0, 0, '膜号')
sheet1.write(0, 1, 'RI值')
sheet1.write(0, 2, 'RI值')
sheet1.write(0, 3, 'RI值')
sheet1.write(0, 4, 'RI值')
sheet1.write(0, 5, 'AVG')
sheet1.write(0, 6, 'CV')
sheet2.write(0, 0, '重错膜号')
sheet2.write(0, 2, '缺失膜号')
sheet2.write(0, 4, '位置偏移')
sheet2.write(0, 6, '绝对值偏低')
sheet2.write(0, 8, 'CV不合格')
sheet2.write(0, 10, '复筛膜号汇总')


In [7]:
monum = 4245 + 1
# 先进行错误膜号的修改
useless_pos = set()  # 输出错号,修改录入出错的情况再进行录入
for i in range(len(d)):
    if i > 0:  # 输出错号,修改录入出错的情况再进行录入
        a = len(d[i - 1][1])
        b = len(d[i][1])
        if a + b == 4:  # 修改录入错误的膜号
            for k in range(a - 1, -1, -1):
                d[i][1].append(d[i - 1][1][k])
                del d[i - 1][1][k]
            d[i - 1][1].append(-1)
            useless_pos.add(d[i - 1][0])
        elif a > b and a + b == 8:  # 修改忘记改号的膜号
            for k in range(a - 1, 3, -1):
                d[i][1].append(d[i - 1][1][k])
                del d[i - 1][1][k]  # d[i - 1][1].remove(d[i - 1][1][k]) remove是对数值去除
        elif a == 8 and b == 1:  # 修改另一种忘记改号的情况
            for k in range(a - 1, 3, -1):
                d[i][1].append(d[i - 1][1][k])
                del d[i - 1][1][k]
            del d[i][1][0]
    if d[i][0]>monum:
        useless_pos.add(d[i][0])
        

In [8]:

# 再进行膜号的分别输出
n = 1
m = 1
j = 0
less_pos = set()
cvfailed = set()
for i in range(index + 1):
    if d[i][0] in wrong_pos:
        sheet2.write(n, 0, d[i][0], styleWrongPos)
        n += 1
    elif not d[i][0] in useless_pos and len(d[i][1]) != 4:
        less_pos.add(d[i][0])
        sheet2.write(n, 0, d[i][0], styleMoreOrLess)
        n += 1
    elif not d[i][0] in useless_pos and not d[i][0] in locfailed and not d[i][0] in absfailed :
        tmp = [float(x) for x in d[i][1]]
        mean = np.mean(tmp)
        std = np.std(tmp) # 用对总体求标准差公式对样本数据求标准差
        cv = std / mean
        if cv<=0.15:
            sheet1.write(m, 0, d[i][0])
            for j in range(len(d[i][1])):
                sheet1.write(m, j + 1, d[i][1][j])
                sheet1.write(m, j + 2, mean)
                sheet1.write(m, j + 3, std / mean)
            m += 1
        else:
            cvfailed.add(d[i][0])

In [9]:

# 找到并输出缺失膜号
lose_pos = set()
monum = 4245 + 1
exist = [False] * monum
n = 1
for data in d:
    if not data[0] in useless_pos and data[0] < monum:
        exist[data[0]] = True
for i in range(len(exist)):
    if not exist[i]:
        if i != 0:
            sheet2.write(n, 2, i)
            lose_pos.add(i)
            n += 1


In [10]:

# 输出不合格膜号
n = 1
for i in locfailed :
    if not i in useless_pos and i not in absfailed and i not in cvfailed:
        sheet2.write(n, 4, i)
        n += 1
n = 1
for i in absfailed:
    if not i in useless_pos and i not in cvfailed:
        sheet2.write(n, 6, i)
        n += 1
n = 1
for i in cvfailed:
    if not i in useless_pos:
        sheet2.write(n, 8, i)
        n += 1
failed = set()
failed = locfailed|absfailed|cvfailed|wrong_pos|lose_pos|less_pos
n = 1
x = 0
l = (len(failed)//10)
for i in failed:
    if i not in useless_pos:
        if n<=l:
            sheet2.write(n, 10 + x, i)
            n += 1
        elif n>l:
            n = 1
            x += 1
            sheet2.write(n, 10 + x, i)
            n += 1


In [11]:

# 导出工作簿
wb.save('筛膜数据处理结果.xls')

# 结束提示
print('请查看文件')


请查看文件
